<a href="https://colab.research.google.com/github/sonalmahawar/extracting-information-frm-financial-reports/blob/main/Extracting_information_from_financial_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import pandas as pd
from nltk.tokenize import RegexpTokenizer, sent_tokenize
import numpy as np
from bs4 import BeautifulSoup
import requests
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df_cik = pd.read_csv("/content/gdrive/MyDrive/cik_list.xlsx - cik_list_ajay.csv")
df_cik.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME
0,3662,SUNBEAM CORP/FL/,199803,6/3/1998,10-K405,edgar/data/3662/0000950170-98-000413.txt
1,3662,SUNBEAM CORP/FL/,199805,15/5/1998,10-Q,edgar/data/3662/0000950170-98-001001.txt
2,3662,SUNBEAM CORP/FL/,199808,13/8/1998,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt
3,3662,SUNBEAM CORP/FL/,199811,12/11/1998,10-K/A,edgar/data/3662/0000950170-98-002145.txt
4,3662,SUNBEAM CORP/FL/,199811,16/11/1998,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt


In [ ]:
y = 'https://www.sec.gov/Archives/'
links = [y+x for x in df_cik['SECFNAME']]

In [ ]:
reports = []
for url in links:
    r = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data, "html.parser")
    reports.append(soup.get_text())

print(f'Total {len(reports)} reports saved')

Total 152 reports saved


In [ ]:
master_df = pd.read_excel('/content/gdrive/MyDrive/LoughranMcDonald_MasterDictionary_2018.xlsx')
master_df.head()

,Word,Sequence Number,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Constraining,Superfluous,Interesting,Modal,Irr_Verb,Harvard_IV,Syllables,Source
0,AARDVARK,1,277,1.480368e-08,1.239377e-08,3.564730e-06,84,0,0,0,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.603287e-10,9.725110e-12,9.863549e-09,1,0,0,0,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,8,4.275431e-10,1.386497e-10,6.225591e-08,7,0,0,0,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,12,6.413147e-10,3.159061e-10,9.383557e-08,12,0,0,0,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,7250,3.874610e-07,3.681624e-07,3.366553e-05,914,0,0,0,0,0,0,0,0,0,0,3,12of12inf


Creating positive and negative word dictionaries

In [ ]:
positive_dictionary = [x for x in master_df[master_df['Positive'] != 0]['Word']]

negative_dictionary = [x for x in master_df[master_df['Negative'] != 0]['Word']]

print(f"Total positve words in dictionary are {len(positive_dictionary)}")
print(f"Total negative words in dictionary are {len(negative_dictionary)}")

Total positve words in dictionary are 354
Total negative words in dictionary are 2355


In [ ]:
df_const = pd.read_excel("/content/gdrive/MyDrive/constraining_dictionary.xlsx")
constraining_words = list(df_const['Word'])


In [ ]:
df_uncert = pd.read_excel("/content/gdrive/MyDrive/uncertainty_dictionary.xlsx")
uncertain_words = list(df_uncert['Word'])

Tokenizer

In [ ]:
def tokenize(text):
    text = re.sub(r'[^A-Za-z]',' ',text.upper())
    tokenized_words = word_tokenize(text)
    return tokenized_words

In [ ]:
df_stopwords = pd.read_csv("/content/gdrive/MyDrive/StopWords_Generic.csv")

In [ ]:
def remove_stopwords(words):
    return [x for x in words if x not in df_stopwords]

In [ ]:
def countfunc(store, words):
    score = 0
    for x in words:
        if(x in store):
            score = score+1
    return score

In [ ]:
def sentiment(score):
    if(score < -0.5):
        return 'Most Negative'
    elif(score >= -0.5 and score < 0):
        return 'Negative'
    elif(score == 0):
        return 'Neutral'
    elif(score > 0 and score < 0.5):
        return 'Positive'
    else:
        return 'Very Positive'
    

Calculating polarity score

In [ ]:
def polarity_score(positive_score, negative_score):
  return (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
 


Calculating subjectivity

In [ ]:
def subjectivity(positive_score, negative_score, num_words):
    return (positive_score+negative_score)/(num_words+ 0.000001)

**Section 2 -Analysis of Readability - Average Sentence Length, percentage of complex words, fog index**

Calculating percentage of complex word 

In [ ]:
def syllable_morethan2(word):
    if(len(word) > 2 and (word[-2:] == 'es' or word[-2:] == 'ed')):
        return False
    
    count =0
    vowels = ['a','e','i','o','u']
    for i in word:
        if(i.lower() in vowels):
            count = count +1
        
    if(count > 2):
        return True
    else:
        return False
    

calculating Fog Index 

In [ ]:
def fog_index_cal(averageSentenceLength, percentageComplexWord):
    return  0.4 * (averageSentenceLength + percentageComplexWord)
    

In [ ]:
sections = ["Management's Discussion and Analysis",
            "Quantitative and Qualitative Disclosures about Market Risk\n",
            "Risk Factors\n"]
caps = [x.upper() for x in sections]

caps.extend(sections)

In [ ]:
col = ['mda','qqdmr','rf']
var = ['positive_score',
      'negative_score',
      'polarity_score',
      'average_sentence_length',
      'percentage_of_complex_words',
      'fog_index',
      'complex_word_count',
      'word_count',
      'uncertainity_score',
      'constraining_score',
      'positive_word_proportion',
      'negative_word_proportion',
      'uncertainity_word_proportion',
      'constraining_word_proportion',
      'constraining_words_whole_report']
for c in col:
    for v in var[:-1]:
        df_cik[c+'_'+v] = 0.0

df_cik[var[-1]] = 0.0


In [ ]:
df_cik.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,mda_percentage_of_complex_words,mda_fog_index,mda_complex_word_count,mda_word_count,mda_uncertainity_score,mda_constraining_score,mda_positive_word_proportion,mda_negative_word_proportion,mda_uncertainity_word_proportion,mda_constraining_word_proportion,qqdmr_positive_score,qqdmr_negative_score,qqdmr_polarity_score,qqdmr_average_sentence_length,qqdmr_percentage_of_complex_words,qqdmr_fog_index,qqdmr_complex_word_count,qqdmr_word_count,qqdmr_uncertainity_score,qqdmr_constraining_score,qqdmr_positive_word_proportion,qqdmr_negative_word_proportion,qqdmr_uncertainity_word_proportion,qqdmr_constraining_word_proportion,rf_positive_score,rf_negative_score,rf_polarity_score,rf_average_sentence_length,rf_percentage_of_complex_words,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainity_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainity_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report
0,3662,SUNBEAM CORP/FL/,199803,6/3/1998,10-K405,edgar/data/3662/0000950170-98-000413.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3662,SUNBEAM CORP/FL/,199805,15/5/1998,10-Q,edgar/data/3662/0000950170-98-001001.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3662,SUNBEAM CORP/FL/,199808,13/8/1998,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3662,SUNBEAM CORP/FL/,199811,12/11/1998,10-K/A,edgar/data/3662/0000950170-98-002145.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3662,SUNBEAM CORP/FL/,199811,16/11/1998,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
section_map = {i:j for i,j in zip(sections, col)}
s_map = {i.upper():j for i,j in zip(sections, col)}

section_map.update(s_map)

In [ ]:
for i in range(len(reports)):
    text = re.sub('Item','ITEM',reports[i])
    for j in caps:
        x = re.search('ITEM\s+[\d]\(*[A-Za-z]*\)*.*\s+\-*\s*'+j, text)
        
        if x:
            start,end = x.span()
            content = (text[start:]).split('ITEM')[1]
            if ('...' not in content) and ('. . .' not in content) and len(content) > 200:
              tokenized_words = tokenize(content)
              #print(f'Total tokenized words are {len(tokenized_words)}')
              words = remove_stopwords(tokenized_words)
              num_words = len(words)
              #print(f'Total words after removing stop words are {len(words)}')
              positive_score = countfunc(positive_dictionary, words)
              negative_score = countfunc(negative_dictionary, words)
              #print(f'Total positive score is {positive_score}')
              #print(f'Total negative score is {negative_score}')
              polarity_scr = polarity_score(positive_score, negative_score)
              #print(polarity_score)
              subjectivity_score = subjectivity(positive_score, negative_score, num_words)
              #print(subjectivity_score)
              #print(sentiment(polarity_score))
                
              sentences = sent_tokenize(content)
              num_sentences = len(sentences)
              average_sentence_length = num_words/num_sentences
              #print(average_sentence_length)
                
                
              num_complexword =0
              uncertainity_score = 0
              constraining_score = 0
              for word in words:
                if(syllable_morethan2(word)):
                  num_complexword = num_complexword+1
                if(word in uncertain_words):
                  uncertainity_score = uncertainity_score+1
                if(word in constraining_words):
                  constraining_score = constraining_score+1
              #print(num_complexword)
              #print(uncertainity_score)
              #print(constraining_score)
                
                
              percentage_complexwords = num_complexword/num_words
              #print(percentage_complexwords)
              fog_indx = fog_index_cal(average_sentence_length, percentage_complexwords)
              #print(fog_indx)
                
              positive_word_proportion = positive_score/num_words
              negative_word_proportion = negative_score/num_words
              uncertainity_word_proportion = uncertainity_score/num_words
              constraining_word_proportion = constraining_score/num_words
                
                #print(positive_word_proportion)
                #print(negative_word_proportion)
                #print(uncertainity_word_proportion)
                #print(constraining_word_proportion)
               
               
              df_cik.at[i,section_map[j]+'_positive_score'] = positive_score
              df_cik.at[i,section_map[j]+'_negative_score'] = negative_score
              df_cik.at[i,section_map[j]+'_polarity_score'] = polarity_score
              df_cik.at[i,section_map[j]+'_average_sentence_length'] = average_sentence_length
              df_cik.at[i,section_map[j]+'_percentage_of_complex_words'] = percentage_complexwords
              df_cik.at[i,section_map[j]+'_fog_index'] = fog_index
              df_cik.at[i,section_map[j]+'_complex_word_count'] = num_complexword
              df_cik.at[i,section_map[j]+'_word_count'] = num_words
              df_cik.at[i,section_map[j]+'_uncertainity_score'] = uncertainity_score
              df_cik.at[i,section_map[j]+'_constraining_score'] = constraining_score
              df_cik.at[i,section_map[j]+'_positive_word_proportion'] = positive_word_proportion
              df_cik.at[i,section_map[j]+'_negative_word_proportion'] = negative_word_proportion
              df_cik.at[i,section_map[j]+'_uncertainity_word_proportion'] = uncertainity_word_proportion
              df_cik.at[i,section_map[j]+'_constraining_word_proportion'] = constraining_word_proportion
                
                
                
                
                
    constraining_words_whole_report = 0
    tokenized_report_words = tokenize(reports[i])
    report_words = remove_stopwords(tokenized_report_words)
    for word in report_words:
        if word in constraining_words:
            constraining_words_whole_report = 1+ constraining_words_whole_report
    #print(constraining_words_whole_repor
    df_cik.at[i,'constraining_words_whole_report'] = constraining_words_whole_report

In [ ]:
df_cik.head()

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,mda_percentage_of_complex_words,mda_fog_index,mda_complex_word_count,mda_word_count,mda_uncertainity_score,mda_constraining_score,mda_positive_word_proportion,mda_negative_word_proportion,mda_uncertainity_word_proportion,mda_constraining_word_proportion,qqdmr_positive_score,qqdmr_negative_score,qqdmr_polarity_score,qqdmr_average_sentence_length,qqdmr_percentage_of_complex_words,qqdmr_fog_index,qqdmr_complex_word_count,qqdmr_word_count,qqdmr_uncertainity_score,qqdmr_constraining_score,qqdmr_positive_word_proportion,qqdmr_negative_word_proportion,qqdmr_uncertainity_word_proportion,qqdmr_constraining_word_proportion,rf_positive_score,rf_negative_score,rf_polarity_score,rf_average_sentence_length,rf_percentage_of_complex_words,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainity_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainity_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report
0,3662,SUNBEAM CORP/FL/,199803,6/3/1998,10-K405,edgar/data/3662/0000950170-98-000413.txt,34.0,14.0,<function polarity_score at 0x7ff47f39d200>,27.736842,0.336243,11.229234,886.0,2635.0,39.0,6.0,0.012903,0.005313,0.014801,0.002277,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1487.0
1,3662,SUNBEAM CORP/FL/,199805,15/5/1998,10-Q,edgar/data/3662/0000950170-98-001001.txt,11.0,56.0,<function polarity_score at 0x7ff47f39d200>,37.520548,0.351588,11.229234,963.0,2739.0,67.0,3.0,0.004016,0.020445,0.024461,0.001095,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1046.0
2,3662,SUNBEAM CORP/FL/,199808,13/8/1998,NT 10-Q,edgar/data/3662/0000950172-98-000783.txt,0.0,0.0,0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,3662,SUNBEAM CORP/FL/,199811,12/11/1998,10-K/A,edgar/data/3662/0000950170-98-002145.txt,0.0,0.0,0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,716.0
4,3662,SUNBEAM CORP/FL/,199811,16/11/1998,NT 10-Q,edgar/data/3662/0000950172-98-001203.txt,0.0,0.0,0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [ ]:
df_cik.to_excel('output.xlsx')